In [ ]:
cd ~/Desktop/MyProjects/moneyManager/

In [ ]:
import pandas as pd
import random
import numpy as np
from datetime import datetime
from marketUtilities.loadTimeSeries import loadTimeSeries
from marketUtilities.loadTimeSeries import getListOfAvailableStocks
from marketUtilities.loadTimeSeries import loadPriceTimeSeries
from marketUtilities.marketSimulator import marketSimulator

In [32]:
def createBalancedTarget(s, memSize=60):
    """ Sweet and elegant.
    """
    d=(s.shift(-20)-s.shift(-1))
    rollingMean = d.rolling(memSize).mean()
    return (d>rollingMean)

def createTarget(s, memSize=60):
    """ Sweet and elegant.
    """
    d=(s.shift(-20)-s.shift(-1))
    return (d>0)


def getPossibleGoodMoves(startDay, endDay, availableStocks):
    priceTimeSeries = loadPriceTimeSeries(startDay, endDay, availableStocks)
    bestPossiblePredictions = {stock: createTarget(priceTimeSeries[stock]) for stock in priceTimeSeries} 
    stockName=availableStocks[0]
    marketMinutes = priceTimeSeries[stockName].index
    possibleGoodMoves=[[] for _ in range(len(marketMinutes))]
    for s in availableStocks:
        for i in range(len(possibleGoodMoves)):
            if bestPossiblePredictions[s][i]:
                 possibleGoodMoves[i].append(s)
    return pd.Series(possibleGoodMoves, index=marketMinutes)# series minute by minute, and good choices.


def getPossibleBadMoves(possibleGoodMoves, availableStocks):
    possibleBadMoves = [[] for _ in range(len(possibleGoodMoves))]
    for s in availableStocks:
        for i in range(len(possibleGoodMoves)):
            if s not in possibleGoodMoves.values[i]:
                possibleBadMoves[i].append(s)
    return pd.Series(possibleBadMoves, index=possibleGoodMoves.index)
    

def simulateStrategy(possibleGoodMoves, possibleBadMoves, modelPrecision, modelRecall):
    numGoodMoves = 100
    numBadMoves = 30
    
    
    
    return # an actual strategy


In [39]:
startDay=datetime(2019,1,1)
endDay=datetime(2020,1,1)
modelPrecision=0.6
modelRecall=0.01
availableStocks=['GS', 'MSFT', 'IBM']
possibleGoodMoves=getPossibleGoodMoves(startDay, endDay, availableStocks)
possibleBadMoves=getPossibleBadMoves(possibleGoodMoves, availableStocks)
simulateStrategy(possibleGoodMoves, possibleBadMoves, modelPrecision, modelRecall)


In [41]:
possibleBadMoves.values

array([list([]), list([]), list([]), ..., list(['GS', 'MSFT', 'IBM']),
       list(['GS', 'MSFT', 'IBM']), list(['GS', 'MSFT', 'IBM'])],
      dtype=object)

In [30]:
possibleGoodMoves.apply(lambda x: len(x)>0).value_counts()

True     75341
False    22399
dtype: int64

In [31]:
possibleBadMoves.apply(lambda x: len(x)==3).value_counts()

False    75341
True     22399
dtype: int64

In [26]:
possibleGoodMoves[60:70]

date
2019-01-02 15:31:00    [GS, MSFT, IBM]
2019-01-02 15:32:00    [GS, MSFT, IBM]
2019-01-02 15:33:00    [GS, MSFT, IBM]
2019-01-02 15:34:00    [GS, MSFT, IBM]
2019-01-02 15:35:00    [GS, MSFT, IBM]
2019-01-02 15:36:00    [GS, MSFT, IBM]
2019-01-02 15:37:00    [GS, MSFT, IBM]
2019-01-02 15:38:00    [GS, MSFT, IBM]
2019-01-02 15:39:00    [GS, MSFT, IBM]
2019-01-02 15:40:00    [GS, MSFT, IBM]
dtype: object

In [28]:
possibleBadMoves[60:70]

date
2019-01-02 15:31:00    []
2019-01-02 15:32:00    []
2019-01-02 15:33:00    []
2019-01-02 15:34:00    []
2019-01-02 15:35:00    []
2019-01-02 15:36:00    []
2019-01-02 15:37:00    []
2019-01-02 15:38:00    []
2019-01-02 15:39:00    []
2019-01-02 15:40:00    []
dtype: object

In [45]:
possibleGoodMoves.apply(len).value_counts()

2    26557
1    24955
3    23829
0    22399
dtype: int64

In [38]:
possibleBadMoves.apply(tuple).value_counts()

()                 23829
(GS, MSFT, IBM)    22399
(GS,)              10397
(MSFT, IBM)         9316
(GS, IBM)           9218
(MSFT,)             9109
(IBM,)              7051
(GS, MSFT)          6421
dtype: int64